# [Python Reference Link](http://www.data8.org/sp20/python-reference.html)
*Run the cell below so that we can set our modules up*

In [ ]:
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Confidence Intervals: Covid Data

In [ ]:
covid_data = Table.read_table("Covid Data_subset.csv")
covid_data

In [ ]:
#Let's consider this data set to be our population, and calculate the death rate of patients with covid.  
#Those with an impossible date for death can be assumed to live in this data set
100*(1 - (covid_data.where('DATE_DIED','9999-99-99').num_rows / covid_data.num_rows))

### Let's play out a scenario

Suppose that we are still in the early stages of the covid pandemic. You work for a hospital that has seen 100 patients so far. You want to know how deadly this illness is to the population. 

In [ ]:
#we proceed by taking a sample of 100 from the covid table. Let's assume that these are the 100 patients we saw.
covid_sample = covid_data.sample(100,with_replacement = False)
covid_sample

Then we can define the following functions to construct an upper and lower bound for the confidence interval

In [ ]:
def confidence_interval_for_death_rate(level):
    stats = make_array()
    for i in np.arange(1000):
        stat = 100*(1 - ((covid_sample.sample()).where('DATE_DIED','9999-99-99').num_rows / covid_sample.num_rows))
        stats = np.append(stats,stat)
    lower_bound = percentile((100 - level)/2,stats)
    upper_bound = percentile( 100 - ((100 - level)/2) , stats)
    return stats, lower_bound, upper_bound

In [ ]:
stats_array,lower_bound,upper_bound = confidence_interval_for_death_rate(95)

In [ ]:
stats_array

In [ ]:
lower_bound

In [ ]:
upper_bound

In [ ]:
Table().with_columns('Statistics of Bootstrap', stats_array).hist()
plt.scatter(lower_bound, -0.002, color='red', s=70);
plt.scatter(upper_bound, -0.002, color='red', s=70);

In [ ]:
def confidence_interval_lower_b(tbl,level,column_label):
    stats = make_array()
    for i in np.arange(10000):
        stat = np.mean(tbl.sample().column(column_label))
        stats = np.append(stats,stat)
    return percentile( (100 - level)/2 , stats)

def confidence_interval_upper_b(tbl,level,column_label):
    stats = make_array()
    for i in np.arange(10000):
        stat = np.mean(tbl.sample().column(column_label))
        stats = np.append(stats,stat)
    return percentile( 100 - ((100 - level)/2) , stats)

In [ ]:
column_to_look_at = 'AGE'
level = 95
print('The confidence interval for',column_to_look_at) 
print('is from',confidence_interval_lower_b(covid_sample,level,column_to_look_at))
print('up to',confidence_interval_upper_b(covid_sample,level,column_to_look_at))

In [ ]:
np.mean(covid_data.column('AGE'))

# Confidence Interval Practice: Math211 Class Survey

In [ ]:
math211_survey = Table.read_table("Math211_FallSurvey.csv")
math211_survey